In [23]:
import snap
import os.path
import itertools
import numpy as np

import util

# Global Variables

In [24]:
__no_label__ = "NO_LABEL"

In [25]:
from enum import Enum
# Constants used in the paper:
# NODE_WEIGHT_EVALUATION("NUMBER_OF_NODES_INSIDE_OF_SN"),
# NODE_WEIGHT_FREQUENCY("NUMBER_OF_SN_INSIDE_OF_HN"),
# NODE_WEIGHT_EVALUATION_AVG("AVG_WEIGHT_ON_SN"),
# PERCENTAGE("PERCENTAGE"),
# NUMBER_OF_INNER_EDGES("NUMBER_OF_INNER_EDGES"),
# LABEL("LABEL"),
# EDGE_RATIO("EDGE_RATIO"),
# GROUPING("GROUPING"),
# REACHABILITY_COUNT("REACH_NUMBER_OF_INNER_PATHS"),
# PATH_OUT("REACH_PATH_OUT_BY_LABEL"),                      # Not used in og repo
# PATH_IN("REACH_PATH_IN_BY_LABEL"),                        # Not used in og repo
# EDGE_WEIGHT("EDGE_WEIGHT"),
# PARTICIPATION_LABEL("PARTICIPATION_LABEL"),
# TRAVERSAL_FRONTIERS("TRAVERSAL_FRONTIERS");

class AttributeConstants(str, Enum):
    # General attributes for all graphs 
    EDGE_LABEL = "EDGE_LABEL" # indicates the edge label, e.g. KNOWS, LIKES...

    # Attributes for the OG graph
    SUPER_NODE_ID = "SUPER_NODE_ID" # indicates the super node id that the node belongs to
    
    # Attributes for the Evaluation graph
    HYPER_NODE_ID = "HYPER_NODE_ID" # indicates the hyper node id that the super node belongs to
    SN_NODE_WEIGHT = "NUMBER_OF_INNER_NODES" # indicates the number of inner nodes the super node has
    SN_EDGE_WEIGHT = "NUMBER_OF_INNER_EDGES" # indicates the number of inner edges the super node has
    
    # Inner-Connectivity - Each of the following attributes will exist for each
    # l-label, i.e., we would have PERCENTAGE_KNOWS, PERCENTAGE_LIKES, for a 
    # graph with l-labels = {KNOWS, LIKES}.
    SN_LABEL_PERCENTAGE = "LABEL_PERCENTAGE_" # percent of l-labeled inner edges
    SN_LABEL_REACH = "LABEL_REACH_" # number of inner node pairs connected with an l-labeled inner edge

    SE_EDGE_WEIGHT = "NUMBER_OF_EDGES" # indicates the number of inner edges the super edge has

    # Attributes for the Merge graph
    HN_AVG_NODE_WEIGHT = "AVG_NUMBER_OF_INNER_NODES" # for a given hyper node, this indicates the average number of inner super nodes its super nodes have

# class NodeAttributeConstants(str, Enum):
    # Compression
    

    # NODE_WEIGHT_AVG = "AVG_WEIGHT_ON_SN"
    
    # NODE_WEIGHT_FREQUENCY = "NUMBER_OF_SN_INSIDE_OF_HN"
    
    # LABEL = "LABEL"
    # GROUPING = "GROUPING"

    # EDGE_RATIO = "EDGE_RATIO"
    
    # PARTICIPATION_LABEL = "PARTICIPATION_LABEL"
    # TRAVERSAL_FRONTIERS = "TRAVERSAL_FRONTIERS"

# class EdgeAttributeConstants(str, Enum):


# class LabelAttributeConstants(str, Enum):
    

# Load datasets

In [26]:
# Generic funtion to read edge file into a network
# TODO: 
# allow any graph type not only snap.TNEANet
# check if binary exists, if so, load that (rename function to make graph/net)
def edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr, tab_separated=False, dump=False):
    context = snap.TTableContext()

    schema = snap.Schema()
    schema.Add(snap.TStrTAttrPr("srcID", snap.atInt))
    schema.Add(snap.TStrTAttrPr("dstID", snap.atInt))
    
    edgeattrv = snap.TStrV()
    for attr, val in edge_attr.items():
        edgeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))
    
    srcnodeattrv = snap.TStrV()
    for attr, val in src_node_attr.items():
        srcnodeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))

    dstnodeattrv = snap.TStrV()
    for attr, val in dst_node_attr.items():
        dstnodeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))

    if tab_separated:
        separator = '\t'
    else:
        separator = ' '

    table = snap.TTable.LoadSS(schema, filename, context, separator, snap.TBool(False))

    # net will be an object of type snap.TNEANet
    net = table.ToNetwork(snap.TNEANet, "srcID", "dstID", srcnodeattrv, dstnodeattrv, edgeattrv, snap.aaFirst)

    if dump:
        net.Dump()

    # Save to binary
    outfile = filename + ".bin"
    FOut = snap.TFOut(outfile)
    table.Save(FOut)
    FOut.Flush()

    return net


def make_residence_hall_network():
    filename = "data/moreno_oz/out.moreno_oz_oz"

    edge_attr = {AttributeConstants.EDGE_LABEL.value : snap.atInt}
    src_node_attr = {}
    dst_node_attr = {}

    return edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr)


def make_pg_paper_network():
    filename = "data/example/pg_paper.txt"

    edge_attr = {AttributeConstants.EDGE_LABEL.value : snap.atStr}
    src_node_attr = {}
    dst_node_attr = {}

    return edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr)

# Grouping

In [27]:
class Session:
    def __init__(self, network):
        self.network = network
        self.labels_freq = {}   # CHECK: do I use the values or only the keys?
        for EI in network.Edges():
            edge_id = EI.GetId()
            attr_values = snap.TStrV()
            network.AttrValueEI(edge_id, attr_values)
            label = attr_values[0]
            self.labels_freq.setdefault(label, 0) + 1
        self.groupings = {}
        self.network.AddIntAttrN(AttributeConstants.SUPER_NODE_ID.value)

    # The 1st attribute of an edge is the edge label
    def __get_edge_ids_per_label(self):
        labels = {}
        for EI in self.network.Edges():
            edge_id = EI.GetId()
            label = self.network.GetStrAttrDatE(
                EI, AttributeConstants.EDGE_LABEL.value)
            labels.setdefault(label, snap.TIntV()).append(edge_id)
        return labels

    def compute_groupings(self):
        labels_edge_ids = self.__get_edge_ids_per_label()
        node_ids_in_groupings = snap.TIntV()
        for label, edge_ids in sorted(labels_edge_ids.items(), 
                                        key=lambda item: len(item[1]), 
                                        reverse=True):
            grouping = self.network.ConvertESubGraph(snap.TNEANet, edge_ids)
            # We cannot use DelNodes; an exception is thrown if we try to remove a 
            # node that is not there. There is not way to continue after the 
            # exception is thrown, hence not all nodes get removed.
            for node_id in node_ids_in_groupings:
                try:
                    grouping.DelNode(node_id)
                except Exception:
                    pass
            for NI in grouping.Nodes():
                node_ids_in_groupings.append(NI.GetId())
            if not grouping.Empty():
                self.groupings[label] = grouping

        # We must place nodes with degree zero in their own grouping
        in_deg_v = self.network.GetNodeInDegV()
        out_deg_v = self.network.GetNodeOutDegV()
        zero_deg_nodes = snap.TIntV()
        for node_id_in_deg in in_deg_v:
            if node_id_in_deg.GetVal2() == 0:
                zero_deg_nodes.Add(node_id_in_deg.GetVal1())
        for node_id_in_deg in out_deg_v:
            if node_id_in_deg.GetVal2() != 0:
                zero_deg_nodes.DelIfIn(node_id_in_deg.GetVal1())
        if not zero_deg_nodes.Empty():
            grouping = self.network.ConvertSubGraph(snap.TNEANet, zero_deg_nodes)
            self.groupings[__no_label__] = grouping


# Evaluation

In [38]:
class SuperNode:
    super_node_id: int
    grouping_label: str
    sub_graph: snap.TNEANet
    
    def __init__(self, super_node_id, grouping_label, sub_graph):
        self.super_node_id = super_node_id
        self.grouping_label = grouping_label
        self.sub_graph = sub_graph


class Evaluation:
    def __init__(self, session):
        self.session = session
        self.super_edge_id_counter = itertools.count(start=1)
        self.super_node_id_counter = itertools.count(start=1)
        self.evaluation_graph = snap.TNEANet.New()

        # Node attributes
        self.evaluation_graph.AddFltAttrN(
            AttributeConstants.SN_NODE_WEIGHT.value)
        self.evaluation_graph.AddFltAttrN(
            AttributeConstants.SN_EDGE_WEIGHT.value)
        for label in self.session.labels_freq.keys():
            self.evaluation_graph.AddFltAttrN(
                AttributeConstants.SN_LABEL_PERCENTAGE.value + label)
            self.evaluation_graph.AddFltAttrN(
                AttributeConstants.SN_LABEL_REACH.value + label)
        self.evaluation_graph.AddIntAttrN(
            AttributeConstants.HYPER_NODE_ID.value)

        # Edge attributes
        self.evaluation_graph.AddStrAttrE(
            AttributeConstants.EDGE_LABEL.value)
        self.evaluation_graph.AddFltAttrE(
            AttributeConstants.SE_EDGE_WEIGHT.value)
        
        self.super_nodes = {}


    def add_super_node(self, inner_node_ids, grouping_label):
        super_node_id = next(self.super_node_id_counter)

        sub_graph = self.session.network.ConvertSubGraph(snap.TNEANet, 
            inner_node_ids)
        super_node = SuperNode(super_node_id, grouping_label, sub_graph)

        self.super_nodes[super_node_id] = super_node

        # Add an attribute to the nodes in the original graph indicating the 
        # the super node they are a part of
        for node_id in inner_node_ids:
            self.session.network.AddIntAttrDatN(
                node_id, super_node_id, 
                AttributeConstants.SUPER_NODE_ID.value)

        return super_node_id


    def evaluate(self):
        for label, grouping in self.session.groupings.items():
            # print("\t+++ Size of %s grouping: %d" % (label, grouping.GetNodes()))
            wccs = grouping.GetWccs()
            for wcc in wccs:
                # print("\t\t+++ Size of subgrouping: %d" % wcc.Len())
                inner_node_ids = snap.TIntV()
                for node_id in wcc:
                    inner_node_ids.Add(node_id)

                super_node_id = self.add_super_node(inner_node_ids, label)


    def build_evaluation_graph(self):
        labels = self.session.labels_freq.keys()
        network = self.session.network

        # Add all super nodes to evaluation graph
        for super_node_id in self.super_nodes.keys():
            self.evaluation_graph.AddNode(super_node_id)

        # Compute super node attributes and its super edges
        for super_node_id, super_node in self.super_nodes.items():
            labels_to_inner_edge_ids = {}
            labels_to_outer_edge_ids = {}
            super_node_connection_counter = {}
            for label in labels:
                labels_to_inner_edge_ids[label] = snap.TIntV()
                labels_to_outer_edge_ids[label] = snap.TIntV()
            
            for EI in super_node.sub_graph.Edges():
                src_node_id = EI.GetSrcNId()
                dst_node_id = EI.GetDstNId()
                edge_id = network.GetEI(src_node_id, dst_node_id).GetId()
                attr_values = snap.TStrV()
                network.AttrValueEI(edge_id, attr_values)
                label = attr_values[0]

                labels_to_inner_edge_ids[label].append(edge_id)
            
            # Add all the stats on the super node in the form of attributes
            # (1) Compression
            # Number of nodes inside super node
            node_weight = super_node.sub_graph.GetNodes()
            self.evaluation_graph.AddFltAttrDatN(super_node_id, node_weight, 
                AttributeConstants.SN_NODE_WEIGHT.value)

            # Number of edges inside super node
            edge_weight = super_node.sub_graph.GetEdges()
            self.evaluation_graph.AddFltAttrDatN(super_node_id, edge_weight, 
                AttributeConstants.SN_EDGE_WEIGHT.value)

            # (2) Inner-Connectivity
            # Store label frequency percentage and calculate reachability
            for label, edge_ids in labels_to_inner_edge_ids.items():
                try:
                    percentage = edge_ids.Len() / edge_weight
                except ZeroDivisionError:
                    percentage = 0
                self.evaluation_graph.AddFltAttrDatN(super_node_id, percentage, 
                    AttributeConstants.SN_LABEL_PERCENTAGE.value + label)
                
                reach = 0
                if edge_ids.Len() > 0:
                    label_sub_graph_sn = network.GetESubGraph(edge_ids)
                    for NI in label_sub_graph_sn.Nodes():
                        node_id = NI.GetId()
                        bfs_tree = label_sub_graph_sn.GetBfsTree(
                            node_id, True, False)
                        reach = reach + bfs_tree.GetEdges()
                self.evaluation_graph.AddFltAttrDatN(super_node_id, reach, 
                    AttributeConstants.SN_LABEL_REACH.value + label)
            
            # (3) Outer-Connectivity
            # computeConcatenationProperties what this function does still 
            # needs implementation
            
            for NI in super_node.sub_graph.Nodes():
                src_node_id = NI.GetId()
                src_super_node_id = network.GetIntAttrDatN(src_node_id,
                    AttributeConstants.SUPER_NODE_ID.value)
                # CHECK - NI.GetOutEdges exists apparently, it returns the edge id
                _, node_ids_at_one_hop = network.GetNodesAtHop(src_node_id, 1, True)
                for dst_node_id in node_ids_at_one_hop:
                    dst_super_node_id = network.GetIntAttrDatN(dst_node_id,
                        AttributeConstants.SUPER_NODE_ID.value)
                    if src_super_node_id != dst_super_node_id:
                        edge_id = network.GetEI(src_node_id, dst_node_id).GetId()
                        attr_values = snap.TStrV()
                        network.AttrValueEI(edge_id, attr_values)
                        label = attr_values[0]
                        super_node_connection_counter[
                            (dst_super_node_id, label)] = (
                                super_node_connection_counter.setdefault((
                                    dst_super_node_id, label), 0) + 1)

            # Add super_edges and their attributes
            for (dst_super_node_id, label), edge_weight in super_node_connection_counter.items():
                # Add super edge to evaluation graph
                super_edge_id = self.evaluation_graph.AddEdge(super_node_id, dst_super_node_id)
                
                # Add label of the super edge
                self.evaluation_graph.AddStrAttrDatE(super_edge_id, 
                    label, AttributeConstants.EDGE_LABEL.value)

                # Number of edges inside super edge
                self.evaluation_graph.AddFltAttrDatE(super_edge_id, 
                    edge_weight, AttributeConstants.SE_EDGE_WEIGHT.value)


# Merge

In [35]:
# CHECK same as super node
class HyperNode:
    hyper_node_id: int
    reach_label: str
    avg_weight: float
    sub_graph: snap.TNEANet

    
    def __init__(self, hyper_node_id, reach_label, avg_weight, sub_graph):
        self.hyper_node_id = hyper_node_id
        self.reach_label = reach_label
        self.avg_weight = avg_weight
        self.sub_graph = sub_graph


# We have two merging strategies
# Abstract merge class
# Two classes for each strategy
# Target merge - sink nodes have in_degree = 0
# Source merge - sink nodes have out_degree = 0
# We are focusing on target merge for now
class Merge:
    def __init__(self, session, evaluation):
        self.session = session
        self.evaluation = evaluation
        self.merge_graph = snap.TNEANet.New()
        self.hyper_edge_id_counter = itertools.count(start=1)
        self.hyper_node_id_counter = itertools.count(start=1)
        self.hyper_nodes = {}
        
        # Node attributes
        self.merge_graph.AddFltAttrN(
            AttributeConstants.HN_AVG_NODE_WEIGHT.value)
        
        # Edge attributes
        self.merge_graph.AddStrAttrE(
            AttributeConstants.EDGE_LABEL.value)
        # self.merge_graph.AddFltAttrE(
        #     EdgeAttributeConstants.EDGE_WEIGHT.value)
        
        self.super_nodes_merged_cnt = 0
    

    # CHECK same as super node
    def add_hyper_node(self, reach_label, avg_node_weight, super_node_ids):
        hyper_node_id = next(self.hyper_node_id_counter)
        self.super_nodes_merged_cnt += super_node_ids.Len()

        sub_graph = self.evaluation.evaluation_graph.ConvertSubGraph(
            snap.TNEANet, super_node_ids)
        hyper_node = HyperNode(
            hyper_node_id, reach_label, avg_node_weight, sub_graph)

        self.hyper_nodes[hyper_node_id] = hyper_node

        # Add an attribute to the nodes in the evaluation graph indicating the 
        # the hyper node they are a part of
        # CHECK maybe add this info to the OG graph too
        for super_node_id in super_node_ids:
            self.evaluation.evaluation_graph.AddIntAttrDatN(
                super_node_id, hyper_node_id, 
                AttributeConstants.HYPER_NODE_ID.value)
        
        return hyper_node_id
        

    def get_highest_reachability(self, node_id):
        reach_label = ""
        reach_label_value = 0
        attr_names = snap.TStrV()
        self.evaluation.evaluation_graph.FltAttrNameNI(node_id, attr_names)
        for attr_name in attr_names:
            if not attr_name.startswith(AttributeConstants.SN_LABEL_REACH.value):
                continue
            attr_value = self.evaluation.evaluation_graph.GetFltAttrDatN(
                node_id, attr_name)
            if attr_value > reach_label_value:
                # CHECK: create separator global and function to get label
                reach_label = attr_name.split("_")[-1]
                reach_label_value = attr_value
        
        return reach_label, reach_label_value


    def get_super_node_merge_info(self):
        super_nodes_merge_info = []

        # returns the in-degree for every node (node_id, degree)
        # node_id_in_degree = self.evaluation.evaluation_graph.GetNodeInDegV()
        for NI in self.evaluation.evaluation_graph.Nodes():
            node_id = NI.GetId()
            in_degree = NI.GetInDeg()
            out_degree = NI.GetOutDeg()
            in_edge_labels = set()
            out_edge_labels = set()

            # CHECK refactor this
            # get in edge labels
            for i in range(in_degree):
                edge_id = NI.GetInEId(i)
                label = self.evaluation.evaluation_graph.GetStrAttrDatE(
                    edge_id, AttributeConstants.EDGE_LABEL.value)
                in_edge_labels.add(label)

            # get out edge labels
            for i in range(out_degree):
                edge_id = NI.GetOutEId(i)
                label = self.evaluation.evaluation_graph.GetStrAttrDatE(
                    edge_id, AttributeConstants.EDGE_LABEL.value)
                out_edge_labels.add(label)
            
            reach_label, reach_label_value = self.get_highest_reachability(
                node_id)
            
            super_nodes_merge_info.append(SuperNodeMergeInfo(
                node_id, in_degree, out_degree, 
                in_edge_labels, out_edge_labels, 
                reach_label, reach_label_value))
        
        return super_nodes_merge_info
    

    # Specify the strategy we want
    def make_hyper_nodes(self, super_nodes_merge_info, verbose=False):
        # we first need to sort the merge info according to the strategy used
        super_nodes_merge_info.sort(key=lambda merge_info: (
            merge_info.reach_label, merge_info.in_edge_labels))
        
        # then we group the objects according to the strategy used
        super_node_groups = itertools.groupby(super_nodes_merge_info, 
            lambda merge_info: (merge_info.reach_label, merge_info.in_edge_labels))

        # CHECK reach_label_value is not used for anything at the moment, 
        # but the OG project had an option to group by that
        for key, group in super_node_groups:
            reach_label = key[0]
            grouped_super_nodes = snap.TIntV()
            total_node_weight = 0

            for merge_info in group:
                node_id = merge_info.node_id
                grouped_super_nodes.append(node_id)
                node_weight = self.evaluation.evaluation_graph.GetFltAttrDatN(
                    node_id, AttributeConstants.SN_NODE_WEIGHT.value)
                self.evaluation.evaluation_graph.GetFltAttrDatN
                total_node_weight += node_weight

            avg_node_weight = total_node_weight / grouped_super_nodes.Len()
            hyper_node_id = self.add_hyper_node(
                reach_label, avg_node_weight, grouped_super_nodes)
        
        if verbose:
            for hn_id, hn in self.hyper_nodes.items():
                print(f"HN_id {hn_id}:")
                print(f"\tReach label: {hn.reach_label};")
                print(f"\tSubgraph nodes: {hn.sub_graph.GetNodes()};")
                for NI in hn.sub_graph.Nodes():
                    print(f"\t\tNode id: {NI.GetId()}.")
                print(f"\tSubgraph edges: {hn.sub_graph.GetEdges()}.")
                for EI in hn.sub_graph.Edges():
                    print(f"\t\tEdge id: {EI.GetId()}; ({EI.GetSrcNId()}) -> ({EI.GetDstNId()})")


    def merge(self):
        # getting super node info in order to merge them
        # we get all the info no matter the strategy used
        super_nodes_merge_info = self.get_super_node_merge_info()
        
        # Adding hyper nodes to the respective class dict
        # Here we need to specify the strategy we want
        self.make_hyper_nodes(super_nodes_merge_info, verbose=True)

        
        
            
        


class SuperNodeMergeInfo():
    def __init__(self, node_id, in_degree, out_degree, in_edge_labels, out_edge_labels, reach_label, reach_label_value):
        self.node_id = node_id
        self.in_degree = in_degree
        self.out_degree = out_degree
        self.in_edge_labels = in_edge_labels
        self.out_edge_labels = out_edge_labels
        self.reach_label = reach_label
        self.reach_label_value = reach_label_value


# Main

In [36]:
print("Loading dataset...")
network = make_pg_paper_network()
# network = make_residence_hall_network()
print("Dataset loaded.")
print("--> Creating session...")
session = Session(network)
print("--> Computing groupings...")
session.compute_groupings()
print("--> Preparing for evaluation...")
evaluation = Evaluation(session)
print("--> Evaluating...")
evaluation.evaluate()
print("--> Building evaluation graph...")
evaluation.build_evaluation_graph()
print("--> Evaluation completed.")
print("\t+++ Number of super nodes: {}".format(
    evaluation.evaluation_graph.GetNodes()))
print("\t+++ Number of super edges: {}".format(
    evaluation.evaluation_graph.GetEdges()))
print("Preparing for merging...")
merge = Merge(session, evaluation)
print("Merging...")
merge.merge()


Loading dataset...
Dataset loaded.
--> Creating session...
--> Computing groupings...
--> Preparing for evaluation...
--> Evaluating...
--> Building evaluation graph...
--> Evaluation completed.
	+++ Number of super nodes: 9
	+++ Number of super edges: 15
Preparing for merging...
Merging...
HN_id 1:
	Reach label: ;
	Subgraph nodes: 2;
		Node id: 8.
		Node id: 9.
	Subgraph edges: 0.
HN_id 2:
	Reach label: l0;
	Subgraph nodes: 1;
		Node id: 1.
	Subgraph edges: 0.
HN_id 3:
	Reach label: l5;
	Subgraph nodes: 5;
		Node id: 2.
		Node id: 3.
		Node id: 4.
		Node id: 6.
		Node id: 7.
	Subgraph edges: 0.
HN_id 4:
	Reach label: l5;
	Subgraph nodes: 1;
		Node id: 5.
	Subgraph edges: 0.


In [37]:
network = make_pg_paper_network()
for NI in network.Nodes():
    out_edges = NI.GetOutEdges()
    # print(f"Node {NI.GetId()}: {NI.GetInEId(0)}")
    # print(type(NI))
    # print(type(out_edges))
    # for x in out_edges:
    #     print(f"Node {NI.GetId()}: {x}")

In [ ]:
print_all_edge_attributes(evaluation.evaluation_graph)

In [9]:
def print_type_attributes(network, id, attr_name_func, attr_value_func):
    attr_names = snap.TStrV()
    getattr(network, attr_name_func)(id, attr_names)
    for attr_name in attr_names:
        val = getattr(network, attr_value_func)(id, attr_name)
        print("--> {}: {}".format(attr_name, val))

def print_all_edge_attributes(network):
    for EI in network.Edges():
        edge_id = EI.GetId()
        src_node_id = EI.GetSrcNId()
        dst_node_id = EI.GetDstNId()
        print("Edge id: {}; ({}) -> ({})".format(
            edge_id, src_node_id,dst_node_id))
        print_type_attributes(network, edge_id, "IntAttrNameEI", 
            "GetIntAttrDatE")
        print_type_attributes(network, edge_id, "FltAttrNameEI",
            "GetFltAttrDatE")
        print_type_attributes(network, edge_id, "StrAttrNameEI",
            "GetStrAttrDatE")

def print_all_node_attributes(network):
    for NI in network.Nodes():
        node_id = NI.GetId()
        print("Node id: {}".format(node_id))
        print_type_attributes(network, node_id, "IntAttrNameNI", 
            "GetIntAttrDatN")
        print_type_attributes(network, node_id, "FltAttrNameNI",
            "GetFltAttrDatN")
        print_type_attributes(network, node_id, "StrAttrNameNI",
            "GetStrAttrDatN")

In [87]:
Graph = evaluation.evaluation_graph
labels = {}
for NI in Graph.Nodes():
    labels[NI.GetId()] = str(NI.GetId())
Graph.DrawGViz(snap.gvlDot, "output.png", " ", labels)

In [ ]:
labels_edge_ids = {}
for EI in pg_paper.Edges():
    edge_id = EI.GetId()
    attr_values = snap.TStrV()
    pg_paper.AttrValueEI(edge_id, attr_values)
    label = attr_values[0]
    labels_edge_ids.setdefault(label, snap.TIntV()).append(edge_id)

sub_graph = pg_paper.GetESubGraph(labels_edge_ids['l0'])

# shortestPath, NIdToDistH = sub_graph.GetShortPathAll(2, IsDir=True)
# for item in NIdToDistH:
#     print(item, NIdToDistH[item])
# print(shortestPath)

reach = 0
for NI in sub_graph.Nodes():
    node_id = NI.GetId()
    bfs_tree = sub_graph.GetBfsTree(node_id, True, False)
    reach = reach + bfs_tree.GetEdges()
    print(node_id, reach)

In [ ]:
EI = pg_paper.GetEI(13,23)
edge_id = EI.GetId()
attr_values = snap.TStrV()
pg_paper.AttrValueEI(edge_id, attr_values)
print(attr_values[0])

In [ ]:
print(residence_hall.GetNodes())
residence_hall_groupings = compute_groupings(residence_hall)

# Archive

In [ ]:
# def merge_sink_nodes(self):
#         super_nodes_highest_reach = {}

#         # returns the in-degree for every node (node_id, degree)
#         node_id_in_degree = self.evaluation.evaluation_graph.GetNodeInDegV()
#         for (node_id, in_degree) in node_id_in_degree:
#             if in_degree != 0:
#                 continue

#             reach_label = ""
#             reach_label_value = -1.0
#             attr_names = snap.TStrV()
#             self.evaluation.evaluation_graph.FltAttrNameNI(node_id, attr_names)
#             for attr_name in attr_names:
#                 if not attr_name.startswith(LabelAttributeConstants.REACH.value):
#                     continue
#                 attr_value = self.evaluation.evaluation_graph.FltAttrValueNI(
#                     node_id, attr_name)
#                 if attr_value > reach_label_value:
#                     # CHECK: create separator global and function to get label
#                     reach_label = attr_name.split("_")[-1]
#                     reach_label_value = attr_value
                    
#             if (reach_label_value > 0):
#                 super_nodes_highest_reach[node_id] = (
#                     reach_label, reach_label_value)

#         super_nodes_by_reach_label = {}
#         # CHECK reach_label_value is not used for anything at the moment, 
#         # but the OG project had an option to group by that
#         for super_node_id, (reach_label, _) in super_nodes_highest_reach.item():
#             super_nodes_by_reach_label.setdefault(
#                 reach_label, snap.TIntV()).append(super_node_id)

#         for reach_label, super_node_ids in super_nodes_by_reach_label.items():
#             hyper_node_id = self.add_hyper_node(reach_label, super_node_ids)
#             self.super_nodes_merged_cnt += super_node_ids.Len()

#              # PERHAPS Add an attribute to the super nodes and the nodes in the
#              # original graph indicating the the hyper node they are a part of